<a href="https://colab.research.google.com/github/entanglement-nak/portfolio-nak/blob/main/DecisionTree_memory_profiler_%E5%BF%85%E8%A6%81%E7%AE%87%E6%89%80%E3%81%AE%E3%81%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from sklearn.utils import resample
!pip install memory_profiler
from memory_profiler import profile

In [ ]:
%load_ext memory_profiler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# データの読み込み
# profile
def load_data(train_path, test_path):
    train_df = pd.read_excel(train_path, header=None)
    test_df = pd.read_excel(test_path, header=None)
    return train_df, test_df

In [ ]:
# データの前処理
# profile
def preprocess_data(train_df, test_df):
    combined_data = pd.concat([train_df, test_df], axis=0)
    combined_data_filled = combined_data.fillna(combined_data.mean())
    combined_data_cleaned = combined_data_filled.replace([np.inf, -np.inf], np.nan).fillna(combined_data_filled.mean())
    X = combined_data_cleaned.drop(combined_data_cleaned.columns[0], axis=1)
    y = combined_data_cleaned[combined_data_cleaned.columns[0]]
    return X, y

In [ ]:
# データの分割とアップサンプリング
# profile
def split_and_upsample(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    df_train = pd.concat([X_train, y_train], axis=1)
    df_normal_train = df_train[df_train[df_train.columns[-1]] == 1]
    df_anomaly_train = df_train[df_train[df_train.columns[-1]] == -1]
    df_anomaly_upsampled = resample(df_anomaly_train, replace=True, n_samples=len(df_normal_train), random_state=123)
    df_upsampled_train = pd.concat([df_normal_train, df_anomaly_upsampled])
    X_train_upsampled = df_upsampled_train.drop(df_upsampled_train.columns[-1], axis=1)
    y_train_upsampled = df_upsampled_train[df_upsampled_train.columns[-1]]
    return X_train_upsampled, y_train_upsampled, X_test, y_test

In [ ]:
# モデルの訓練 - 決定木
# profile
def train_model(X_train_upsampled, y_train_upsampled):
    decision_tree = DecisionTreeClassifier(max_depth=17)
    decision_tree.fit(X_train_upsampled, y_train_upsampled)
    return decision_tree

In [ ]:
import time

# 予測と評価（時間計測付き）
def predict_and_evaluate(model, X_test, y_test):
    # 予測時間の計測を開始
    start_time = time.time()
    y_pred = model.predict(X_test)
    # 予測時間の計測を終了
    end_time = time.time()

    # 予測にかかった時間を計算
    elapsed_time = end_time - start_time

    # 評価指標の計算
    accuracy = accuracy_score(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    weighted_f1 = f1_score(y_test, y_pred, average='weighted')
    mcc = matthews_corrcoef(y_test, y_pred)

    return accuracy, macro_f1, weighted_f1, mcc, elapsed_time

In [ ]:
# メイン関数（時間計測を含む）
def main():
    # データパス
    train_path = r"/content/drive/MyDrive/Wafer/Wafer_TRAIN.xlsx"
    test_path = r"/content/drive/MyDrive/Wafer/Wafer_TEST.xlsx"

    # データの読み込み
    train_df, test_df = load_data(train_path, test_path)

    # データの前処理
    X, y = preprocess_data(train_df, test_df)

    # データの分割とアップサンプリング
    X_train_upsampled, y_train_upsampled, X_test, y_test = split_and_upsample(X, y)

    # モデルの訓練
    model = train_model(X_train_upsampled, y_train_upsampled)

    # 予測と評価
    accuracy, macro_f1, weighted_f1, mcc, elapsed_time = predict_and_evaluate(model, X_test, y_test)

    # 結果の出力
    print(f"Accuracy: {accuracy}, Macro F1: {macro_f1}, Weighted F1: {weighted_f1}, MCC: {mcc}")
    print(f"予測にかかった時間: {elapsed_time} 秒")

In [ ]:
# 10回計測する
for i in range(10):
    print(f"{i+1}回目の計測")
    %memit main()
    print("\n")

1回目の計測
Accuracy: 0.9893023255813953, Macro F1: 0.9714012904986933, Weighted F1: 0.9894812306939209, MCC: 0.9436555274484599
予測にかかった時間: 0.003215789794921875 秒
peak memory: 316.11 MiB, increment: 47.31 MiB


2回目の計測
Accuracy: 0.9893023255813953, Macro F1: 0.9715133196113368, Weighted F1: 0.9895012609041918, MCC: 0.944084254477717
予測にかかった時間: 0.0046138763427734375 秒
peak memory: 316.21 MiB, increment: 47.05 MiB


3回目の計測
Accuracy: 0.9916279069767442, Macro F1: 0.977485391684372, Weighted F1: 0.991744101351212, MCC: 0.9555620054342758
予測にかかった時間: 0.0032575130462646484 秒
peak memory: 314.68 MiB, increment: 45.47 MiB


4回目の計測
Accuracy: 0.9893023255813953, Macro F1: 0.9715133196113368, Weighted F1: 0.9895012609041918, MCC: 0.944084254477717
予測にかかった時間: 0.0032165050506591797 秒
peak memory: 319.16 MiB, increment: 49.95 MiB


5回目の計測
Accuracy: 0.9916279069767442, Macro F1: 0.9775742656875834, Weighted F1: 0.9917600211134225, MCC: 0.9559149525084752
予測にかかった時間: 0.003326416015625 秒
peak memory: 319.96 Mi